In [1]:
import pandas as pd
import numpy as np
import json

## 导入三元组以及概念上位信息，处理成完整的语义网络

In [12]:
#isA关系加入csv中，为neo4J准备

#df_rel = pd.read_csv('CCFD_triples_isA_added.csv')
#df_entity = pd.read_csv('CCFD_entities_with_tail.csv')



#isA关系和其他关系分开，为大宽表准备

#df_rel_raw = pd.read_csv('00_Raw Data/Wenyue/From_Wenyeu/CCFD_triples.csv')
df_rel_raw = pd.read_csv('00_Raw Data/All concepts and features.csv')
df_concept_info = pd.read_csv('00_Raw Data/CCFD_concept_info.csv')
df_concept_info.head()

In [17]:
# join 三元组及每个entity的属性信息

df_complete = df_rel_raw.join(df_concept_info.set_index('Id'), on='Concept').drop(["Unnamed: 4", "Label", "concept", "superior category", "subcategory", "familiarity", "concretness", "coverage"], axis=1)
df_complete.head()

,Concept,Rel,Feature,Frequency,上级类别,下级类别
0,安全带,可以,可以-保护,27.096774,人造物,工具
1,安全带,是,是-带子,22.258065,人造物,工具
2,安全带,其他,其他-车,17.419355,人造物,工具
3,安全带,是,是-安保的,16.451613,人造物,工具
4,安全带,可以,可以-保护人,15.483871,人造物,工具


In [68]:
# df_complete.to_csv('01_Processed Data/Complete-Data.csv',index=False)
df_complete = pd.read_csv('01_Processed Data/Complete-Data.csv')

## 模拟患者语义网络

In [54]:
df_concepts = pd.read_csv("00_Raw Data/CCFD_concept_info.csv")
df_concepts.head()

,Id,Label,concept,familiarity,concretness,上级类别,superior category,下级类别,subcategory,coverage
0,安全带,安全带,Safety belt,9.00,9.0,人造物,artificial object,工具,tool,0.851345
1,安全帽,安全帽,Safety hat,8.33,9.0,人造物,artificial object,工具,tool,0.717619
2,安慰奶嘴,安慰奶嘴,pacifier,9.00,9.0,人造物,artificial object,餐具,tableware,0.663182
3,鹌鹑,鹌鹑,quail,7.00,9.0,动物,animal,鸟,bird,0.731581
4,鹌鹑蛋,鹌鹑蛋,Quail egg,9.00,9.0,食物,food,固态干粮,solid food,0.741644


In [49]:
subcategories = list(df_concepts.下级类别.unique())

In [64]:
for subcategory in subcategories:
    print(list(df_concepts[df_concepts['下级类别']==subcategory].sort_values(by=['familiarity', 'coverage','concretness'],ascending=False).head()["Label"]),",")

['安全带', '洗脸盆', '针', '钢笔', '锄头'] ,
['餐刀', '汤碗', '碗', '茶壶', '汤匙'] ,
['鹰', '秃鹰', '燕子', '喜鹊', '野鸡'] ,
['鸡蛋', '巧克力冰淇淋', '大饼', '炸酱面', '三明治'] ,
['台式电脑', '灯', '复印机', '手机', '电动牙刷'] ,
['橙汁', '葡萄汁', '牛奶', '豆浆', '全脂牛奶'] ,
['轿车', '房车', '公交车', '马车', '有轨电车'] ,
['衬衫', '跑鞋', '夹克', '风衣', '手套'] ,
['老虎', '豹子', '狮子', '麋鹿', '狼'] ,
['高尔夫球场', '茅草屋', '电话亭', '假山', '石碑'] ,
['瓜', '番茄', '西红柿', '小白菜', '丝瓜'] ,
['水蜜桃', '柑橘', '西瓜', '蜜桃', '葡萄'] ,
['松树', '竹子', '槐树', '垂柳', '柳树'] ,
['豆豉', '辣椒酱', '白糖', '番茄酱', '味精'] ,
['蜥蜴', '瓢虫', '蜻蜓', '蜘蛛', '萤火虫'] ,
['水晶', '星星', '地球', '煤气', '瀑布'] ,
['郁金香', '莲花', '菊花', '捕蝇草', '雏菊'] ,
['乒乓球', '球拍', '羽毛球拍', '排球', '羽毛球'] ,
['沙发', '弹簧床垫', '毛毯', '旋转椅', '床垫'] ,
['鲨鱼', '鲸鱼', '螺蛳', '水母', '牡蛎'] ,
['手术刀', '纱布', '体温计', '呼吸机', '创可贴'] ,
['骰子', '气球', '水晶球', '水枪', '摩天轮'] ,
['腿', '羽毛', '胡须', '嘴唇', '耳朵'] ,
['弹簧刀', '刺刀', '刀', '飞镖', '坦克'] ,
['音乐盒', '钢琴', '电吉他', '小提琴', '扩音器'] ,
['豪华客轮', '游轮', '独木舟', '游艇', '皮艇'] ,
['牛排', '生鱼片', '鸭肉', '鱼丸', '肉丸'] ,
['飞机', '直升机', '航天飞机', '热气球', '太空舱'] ,


In [65]:
## 提示词
cues = [
#     ['耳朵','手掌', '脚后跟','肩膀','牙齿','鼻子'],
#     ['鸽子','麻雀', '燕子','天鹅','喜鹊','猫头鹰'],
#     ['猴子','狮子', '兔子','老虎','老鼠','奶牛'],
#     ['苹果','香蕉', '葡萄','甘蔗','柑橘','西瓜'],
#     ['萝卜','土豆', '白菜','黄瓜','韭菜','茄子'],
#     ['直升机','火车', '帆船','卡车','轿车','摩托车'],
#     ['茅草屋','人行天桥', '车库','立交桥','高速公路','喷泉'],
#     ['衬衫','毛衣', '夹克','袜子','围巾','裤子'],
#     ['沙发','椅子', '餐桌','书柜','马桶','冰箱'],
#     ['扫帚','雨伞', '剪刀','纱布','拐杖','筷子']
    
    ['安全带', '洗脸盆', '针', '钢笔', '锄头'] ,
    ['餐刀', '汤碗', '碗', '茶壶', '汤匙'] ,
    ['鹰', '秃鹰', '燕子', '喜鹊', '野鸡'] ,
    ['鸡蛋', '巧克力冰淇淋', '大饼', '炸酱面', '三明治'] ,
    ['台式电脑', '灯', '复印机', '手机', '电动牙刷'] ,
    ['橙汁', '葡萄汁', '牛奶', '豆浆', '全脂牛奶'] ,
    ['轿车', '房车', '公交车', '马车', '有轨电车'] ,
    ['衬衫', '跑鞋', '夹克', '风衣', '手套'] ,
    ['老虎', '豹子', '狮子', '麋鹿', '狼'] ,
    ['高尔夫球场', '茅草屋', '电话亭', '假山', '石碑'] ,
    ['瓜', '番茄', '西红柿', '小白菜', '丝瓜'] ,
    ['水蜜桃', '柑橘', '西瓜', '蜜桃', '葡萄'] ,
    ['松树', '竹子', '槐树', '垂柳', '柳树'] ,
    ['豆豉', '辣椒酱', '白糖', '番茄酱', '味精'] ,
    ['蜥蜴', '瓢虫', '蜻蜓', '蜘蛛', '萤火虫'] ,
    ['水晶', '星星', '地球', '煤气', '瀑布'] ,
    ['郁金香', '莲花', '菊花', '捕蝇草', '雏菊'] ,
    ['乒乓球', '球拍', '羽毛球拍', '排球', '羽毛球'] ,
    ['沙发', '弹簧床垫', '毛毯', '旋转椅', '床垫'] ,
    ['鲨鱼', '鲸鱼', '螺蛳', '水母', '牡蛎'] ,
    ['手术刀', '纱布', '体温计', '呼吸机', '创可贴'] ,
    ['骰子', '气球', '水晶球', '水枪', '摩天轮'] ,
    ['腿', '羽毛', '胡须', '嘴唇', '耳朵'] ,
    ['弹簧刀', '刺刀', '刀', '飞镖', '坦克'] ,
    ['音乐盒', '钢琴', '电吉他', '小提琴', '扩音器'] ,
    ['豪华客轮', '游轮', '独木舟', '游艇', '皮艇'] ,
    ['牛排', '生鱼片', '鸭肉', '鱼丸', '肉丸'] ,
    ['飞机', '直升机', '航天飞机', '热气球', '太空舱']
]

cues = sum(cues,[])

In [66]:
len(cues)

140

In [69]:
## 选择提示词相关的所有语义特征，作为个人语义网络数据

#检验提示词是否在数据库中
#len(df_rel[df_rel['head']=="脚跟"])

for cue in cues:
    if len(df_complete[df_complete['Concept']==cue]) == 0:
        print(cue)
    else:
        print(df_complete[df_complete['Concept']==cue])
        
df_complete_individual = df_complete[df_complete['Concept'].isin(cues)]

   Concept Rel  Feature  Frequency 上级类别 下级类别
0      安全带  可以    可以-保护  27.096774  人造物   工具
1      安全带   是     是-带子  22.258065  人造物   工具
2      安全带  其他     其他-车  17.419355  人造物   工具
3      安全带   是    是-安保的  16.451613  人造物   工具
4      安全带  可以   可以-保护人  15.483871  人造物   工具
5      安全带  需要     需要-系  13.548387  人造物   工具
6      安全带   像     像-绳子   8.709677  人造物   工具
7      安全带   有      有-扣   8.709677  人造物   工具
8      安全带   是     是-长的   6.774194  人造物   工具
9      安全带  可以    可以-固定   5.806452  人造物   工具
10     安全带  需要     需要-扣   5.806452  人造物   工具
11     安全带   有     有-扣子   5.806452  人造物   工具
12     安全带  可以     可以-系   4.838710  人造物   工具
13     安全带   是     是-部件   4.838710  人造物   工具
14     安全带   是    是-条状的   4.838710  人造物   工具
15     安全带   有      有-布   4.838710  人造物   工具
16     安全带   是    是-必需品   3.870968  人造物   工具
17     安全带   是     是-工具   3.870968  人造物   工具
18     安全带   是    是-黑色的   3.870968  人造物   工具
19     安全带  需要    需要-佩戴   3.870968  人造物   工具
20     安全带   有     有-弹性   3.870968  人造物   工具
21     安全带

39195    台式电脑   有   有-多种品牌        2.0  人造物   电器
     Concept Rel Feature  Frequency 上级类别 下级类别
8449       灯  需要    需要-电  20.357143  人造物   电器
8450       灯   有    有-灯泡   9.642857  人造物   电器
8451       灯   有    有-灯罩   9.642857  人造物   电器
8452       灯   有    有-罩子   9.642857  人造物   电器
8453       灯  可以   可以-照明   8.571429  人造物   电器
8454       灯   是    是-工具   8.571429  人造物   电器
8455       灯   是    是-亮的   8.571429  人造物   电器
8456       灯   有    有-灯丝   7.500000  人造物   电器
8457       灯   有     有-光   7.500000  人造物   电器
8458       灯  可以   可以-发光   6.428571  人造物   电器
8459       灯   像    像-太阳   6.428571  人造物   电器
8460       灯  可以   可以-发热   5.357143  人造物   电器
8461       灯   像    像-蜡烛   5.357143  人造物   电器
8462       灯   有    有-电线   5.357143  人造物   电器
8463       灯   是    是-用具   4.285714  人造物   电器
8464       灯   有    有-玻璃   4.285714  人造物   电器
8465       灯   有  有-多种种类   4.285714  人造物   电器
8466       灯   是    是-电器   3.214286  人造物   电器
8467       灯   是    是-家具   3.214286  人造物   电器
8468       灯   像    像-灯笼   3.214

      Concept Rel        Feature  Frequency 上级类别 下级类别
37010      手套  可以          可以-保暖  18.620690  人造物   服饰
37011      手套   像            像-手  14.482759  人造物   服饰
37012      手套  可以           可以-戴  12.413793  人造物   服饰
37013      手套  可以          可以-保护  11.379310  人造物   服饰
37014      手套  需要          需要-清洗  10.344828  人造物   服饰
37015      手套  可以         可以-保护手   9.310345  人造物   服饰
37016      手套   像           像-袜子   9.310345  人造物   服饰
37017      手套  其他           其他-手   7.241379  人造物   服饰
37018      手套  需要          需要-制造   7.241379  人造物   服饰
37019      手套  可以           可以-穿   6.206897  人造物   服饰
37020      手套   是          是-安保的   6.206897  人造物   服饰
37021      手套   有            有-布   6.206897  人造物   服饰
37022      手套   有         有-多种材质   6.206897  人造物   服饰
37023      手套   有         有-多种颜色   6.206897  人造物   服饰
37024      手套   是         是-生活用品   5.172414  人造物   服饰
37025      手套   是           是-衣服   5.172414  人造物   服饰
37026      手套   有           有-毛线   5.172414  人造物   服饰
37027      手套   是           

28921      蜜桃  需要    需要-削皮        2.0   植物   水果
      Concept Rel Feature  Frequency 上级类别 下级类别
32384      葡萄   是    是-水果  30.000000   植物   水果
32385      葡萄   是    是-食物  24.193548   植物   水果
32386      葡萄   有     有-籽  13.548387   植物   水果
32387      葡萄   是    是-甜的  12.580645   植物   水果
32388      葡萄  需要    需要-水  12.580645   植物   水果
32389      葡萄   是   是-紫色的  11.612903   植物   水果
32390      葡萄  需要   需要-种植  11.612903   植物   水果
32391      葡萄   有    有-果肉  10.645161   植物   水果
32392      葡萄   有     有-皮  10.645161   植物   水果
32393      葡萄  可以   可以-酿酒   8.709677   植物   水果
32394      葡萄  需要   需要-采摘   8.709677   植物   水果
32395      葡萄  需要   需要-阳光   8.709677   植物   水果
32396      葡萄   是    是-植物   7.741935   植物   水果
32397      葡萄  需要   需要-清洗   7.741935   植物   水果
32398      葡萄   是   是-绿色的   6.774194   植物   水果
32399      葡萄   是    是-酸的   6.774194   植物   水果
32400      葡萄  需要   需要-空气   6.774194   植物   水果
32401      葡萄  需要    需要-土   6.774194   植物   水果
32402      葡萄   有  有-多种颜色   6.774194   植物   水果
32403      葡

      Concept Rel   Feature  Frequency 上级类别 下级类别
32630      瀑布   是       是-水       27.0  自然物  自然物
32631      瀑布  可以     可以-观赏       22.0  自然物  自然物
32632      瀑布   是      是-景观       16.0  自然物  自然物
32633      瀑布   是     是-自然的       10.0  自然物  自然物
32634      瀑布   是     是-壮观的        6.0  自然物  自然物
32635      瀑布   像      像-窗帘        5.0  自然物  自然物
32636      瀑布   有      有-落差        5.0  自然物  自然物
32637      瀑布  可以     可以-发电        4.0  自然物  自然物
32638      瀑布  可以     可以-拍照        4.0  自然物  自然物
32639      瀑布  需要    需要-高度差        4.0  自然物  自然物
32640      瀑布  可以     可以-发声        3.0  自然物  自然物
32641      瀑布   是      是-风景        3.0  自然物  自然物
32642      瀑布   是     是-高处的        3.0  自然物  自然物
32643      瀑布   是     是-流动的        3.0  自然物  自然物
32644      瀑布   像       像-布        3.0  自然物  自然物
32645      瀑布   像      像-水帘        3.0  自然物  自然物
32646      瀑布   像      像-银河        3.0  自然物  自然物
32647      瀑布  需要     需要-爱护        3.0  自然物  自然物
32648      瀑布   有    有-多种型号        3.0  自然物  自然物
32649      瀑布   有   

38127      水母   有     有-须        2.0   动物  水生动物
      Concept Rel Feature  Frequency 上级类别  下级类别
29578      牡蛎   是    是-食物  28.064516   动物  水生动物
29579      牡蛎   有     有-壳  27.096774   动物  水生动物
29580      牡蛎   是   是-海里的  20.322581   动物  水生动物
29581      牡蛎   是    是-动物  14.516129   动物  水生动物
29582      牡蛎   有     有-肉  12.580645   动物  水生动物
29583      牡蛎   是    是-生物   9.677419   动物  水生动物
29584      牡蛎   是    是-软的   8.709677   动物  水生动物
29585      牡蛎  需要    需要-水   7.741935   动物  水生动物
29586      牡蛎   是    是-海鲜   6.774194   动物  水生动物
29587      牡蛎   是    是-贝类   5.806452   动物  水生动物
29588      牡蛎   像    像-扇贝   5.806452   动物  水生动物
29589      牡蛎  需要   需要-食物   5.806452   动物  水生动物
29590      牡蛎  需要   需要-海水   4.838710   动物  水生动物
29591      牡蛎   是   是-海产品   3.870968   动物  水生动物
29592      牡蛎   是   是-好吃的   3.870968   动物  水生动物
29593      牡蛎   是     是-药   3.870968   动物  水生动物
29594      牡蛎   像    像-生蚝   3.870968   动物  水生动物
29595      牡蛎  需要   需要-捕捞   3.870968   动物  水生动物
29596      牡蛎  需要   需要-养殖   3.870968   动

11878      飞镖   有  有-多种颜色   1.935484  人造物   武器
      Concept Rel  Feature  Frequency 上级类别 下级类别
39490      坦克   是     是-武器  23.225806  人造物   武器
39491      坦克   有     有-履带  15.483871  人造物   武器
39492      坦克   是      是-车  10.645161  人造物   武器
39493      坦克   有     有-轮子  10.645161  人造物   武器
39494      坦克  可以    可以-打仗   8.709677  人造物   武器
39495      坦克   是     是-战争   7.741935  人造物   武器
39496      坦克   有    有-驾驶室   6.774194  人造物   武器
39497      坦克   有     有-大炮   5.806452  人造物   武器
39498      坦克  可以     可以-开   4.838710  人造物   武器
39499      坦克   是     是-大的   4.838710  人造物   武器
39500      坦克  需要    需要-司机   4.838710  人造物   武器
39501      坦克   有     有-炮弹   4.838710  人造物   武器
39502      坦克   有     有-炮筒   4.838710  人造物   武器
39503      坦克   是     是-硬的   3.870968  人造物   武器
39504      坦克   像    像-装甲车   3.870968  人造物   武器
39505      坦克  需要    需要-驾驶   3.870968  人造物   武器
39506      坦克  需要    需要-瞄准   3.870968  人造物   武器
39507      坦克  需要     需要-油   3.870968  人造物   武器
39508      坦克   有     有-火炮   3.870968  人造

16638    航天飞机   有      有-助推器   1.935484  交通工具   飞机
      Concept Rel Feature  Frequency  上级类别 下级类别
34315     热气球   是    是-气球  23.225806  交通工具   飞机
34316     热气球  可以    可以-飞  19.354839  交通工具   飞机
34317     热气球   是    是-大的   9.677419  交通工具   飞机
34318     热气球   是    是-工具   8.709677  交通工具   飞机
34319     热气球   有    有-吊篮   8.709677  交通工具   飞机
34320     热气球  可以   可以-旅行   5.806452  交通工具   飞机
34321     热气球   是    是-器械   5.806452  交通工具   飞机
34322     热气球   像   像-降落伞   5.806452  交通工具   飞机
34323     热气球  需要   需要-燃料   5.806452  交通工具   飞机
34324     热气球   有     有-火   5.806452  交通工具   飞机
34325     热气球   是   是-飞行器   4.838710  交通工具   飞机
34326     热气球  需要   需要-点火   4.838710  交通工具   飞机
34327     热气球  需要    需要-风   4.838710  交通工具   飞机
34328     热气球   有    有-气体   4.838710  交通工具   飞机
34329     热气球   像    像-飞机   3.870968  交通工具   飞机
34330     热气球  需要   需要-安全   3.870968  交通工具   飞机
34331     热气球  需要   需要-加热   3.870968  交通工具   飞机
34332     热气球   有  有-多种颜色   3.870968  交通工具   飞机
34333     热气球   有    有-篮子   3.870968 

In [74]:
## 随机采样，模拟受损的个人语义网络

def Sample_PWA(df, sample_rate, mark_col):
    np.random.seed(10)
    remove_n = round(len(df)*sample_rate)
    drop_indices = np.random.choice(df.index, remove_n, replace=False)
    
    df_sample = df.copy()
    df_sample[mark_col] = 0
    for row in df_sample.iterrows():
        if row[0] in drop_indices:
            df_sample.loc[row[0],mark_col] = 1
        else:
            pass

    return df_sample

In [77]:
df_sample = Sample_PWA(df_complete_individual, 0.7, 'impair')

In [78]:
df_sample

,Concept,Rel,Feature,Frequency,上级类别,下级类别,impair
0,安全带,可以,可以-保护,27.096774,人造物,工具,0
1,安全带,是,是-带子,22.258065,人造物,工具,1
2,安全带,其他,其他-车,17.419355,人造物,工具,0
3,安全带,是,是-安保的,16.451613,人造物,工具,0
4,安全带,可以,可以-保护人,15.483871,人造物,工具,1
...,...,...,...,...,...,...,...
51921,嘴唇,有,有-上下,2.068966,身体部位,身体部位,0
51922,嘴唇,有,有-上嘴唇,2.068966,身体部位,身体部位,1
51923,嘴唇,有,有-下嘴唇,2.068966,身体部位,身体部位,0
51924,嘴唇,有,有-血液,2.068966,身体部位,身体部位,0


In [80]:
#为neo4j存储
#df_impair_individual.to_csv('CCFD_60_words_0.7_sampleRate.csv',index=False)

#为大宽表存储
# df_sample.to_csv('01_Processed Data/PWA_Minor.csv', index=False)
df_impair_individual = pd.read_csv('01_Processed Data/PWA_Minor.csv')

## 从完整网络中抽取关联网络

In [27]:
## 抽取关联网络
def Extract_Association_Network(df, entity_col, association_col, strength_thres):
    Source = []
    Target = []
    Strength = []
    
    entities = list(df[entity_col].unique())
    
    for i in range(len(entities)):
        for j in range(1, (len(entities) - i)):
            listA = list(df[df[entity_col]==entities[i]][association_col].unique())
            listB = list(df[df[entity_col]==entities[i+j]][association_col].unique())
            strength = len(set(listA)&set(listB))
            if strength > strength_thres:
                #print(features[i],features[i+j],features)
                Source.append(entities[i])
                Target.append(entities[i+j])
                Strength.append(strength)
                
    data_AssociationNetwork = {'Source': Source, 'Target': Target, 'Strength': Strength}
    df_AssociationNetwork = pd.DataFrame(data = data_AssociationNetwork)
    return df_AssociationNetwork

In [81]:
df_impair_individual_concept_network = Extract_Association_Network(df_impair_individual, 'Concept', 'Feature', 5)
df_impair_individual_concept_network.head()

,Source,Target,Strength
0,安全带,手机,6
1,白糖,橙汁,9
2,白糖,大饼,6
3,白糖,豆豉,9
4,白糖,豆浆,8


In [28]:
df_impair_individual_feature_network = Extract_Association_Network(df_impair_individual, 'Feature', 'Concept', 5)
df_impair_individual_feature_network.head()

,Source,Target,Strength
0,是-蔬菜,是-植物,6
1,是-蔬菜,需要-水,6
2,是-蔬菜,需要-阳光,6
3,是-蔬菜,是-食物,6
4,是-蔬菜,需要-空气,6


In [29]:
df_impair_individual_feature_network.to_csv('01_Processed Data/SemanticFeature_Link_PWA-Minor.csv')

In [88]:
df.to_csv('01_Processed Data/SemanticFeature_Link.csv',index=False)
#df_ConceptNetwork.to_csv('02_Graph Data/Concept_Link.csv',index=False)

## Node_Link CSV to NodeTrix Json

In [150]:
# Nodetrix json format

# "nodes":[
#     {"name":"Myriel","group":1},
#         ...]
# "links":[
#     {"source":1,"target":0,"value":1},
#         ...]

In [84]:
# 为了适应NodeTrix/G6等不同库的数据格式需求，设置两个参数
# link_object_name: "links" / "edges" 
# node_content_mode: "index" / "name"

def Node_Link_CSV2Json(df_link, df_node, node_name_col, node_group_col, link_object_name, node_content_mode):
    
    data = {
        "nodes": [],
        link_object_name: []
    }
    groups = []

    nodes = list(set(list(df_link.Source)+list(df_link.Target)))
    for node in nodes: 
        df_selected_node = df_node[df_node[node_name_col]==node]
        group = list(df_selected_node[node_group_col])[0]
        if group not in groups:
            groups.append(group)
        new_node = {        
            "id":list(df_selected_node[node_name_col])[0],
            "group":groups.index(group)
        } 
        if new_node not in data['nodes']:
            data['nodes'].append(new_node)


    for row in df_link.iterrows():
        
        if node_content_mode == "index":
            new_link = {
                "source":nodes.index(row[1]['Source']),
                "target":nodes.index(row[1]['Target']),
                "value":int(row[1]['Strength'])
            }
        elif node_content_mode == "name":
            new_link = {
                "source":row[1]['Source'],
                "target":row[1]['Target'],
                "value":int(row[1]['Strength'])
            }
            
        if new_link not in data[link_object_name]:
            data[link_object_name].append(new_link)
            
    print("nodes: "+str(len(data['nodes'])))
    print("links: "+str(len(data[link_object_name])))
            
    return data

In [85]:
df_node = pd.read_csv('00_Raw Data/CCFD_concept_info.csv')
df_node.head()

,Id,Label,concept,familiarity,concretness,上级类别,superior category,下级类别,subcategory,coverage
0,安全带,安全带,Safety belt,9.00,9.0,人造物,artificial object,工具,tool,0.851345
1,安全帽,安全帽,Safety hat,8.33,9.0,人造物,artificial object,工具,tool,0.717619
2,安慰奶嘴,安慰奶嘴,pacifier,9.00,9.0,人造物,artificial object,餐具,tableware,0.663182
3,鹌鹑,鹌鹑,quail,7.00,9.0,动物,animal,鸟,bird,0.731581
4,鹌鹑蛋,鹌鹑蛋,Quail egg,9.00,9.0,食物,food,固态干粮,solid food,0.741644


In [25]:
df_concept_network = pd.read_csv('02_Graph/Concept_Link.csv')
df_concept_network.head()

,Source,Target,Strength
0,安全带,安全帽,7
1,安全带,按键,5
2,安全带,按摩器,2
3,安全带,按钮,6
4,安全带,八哥,1


In [86]:
data = Node_Link_CSV2Json(df_impair_individual_concept_network,df_node,"Label","下级类别", "edges", "name")
%%time

nodes: 139
links: 1407


UsageError: Line magic function `%%time` not found.


In [38]:
data = Node_Link_CSV2Json(test_2,df_node,"Feature","Rel", "edges", "name")
%%time

nodes: 82
links: 577


In [87]:
with open("04_G6/CCFD_G6_Concept_PWA-Minor.json_139", "w") as f: 
    json.dump(data, f, ensure_ascii=False)